In [1]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import huggingface_hub
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role
from tqdm.auto import tqdm
import fsspec


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [3]:
hf_api_key = utils.get_secret(region_name=config.AWS_REGION, secret_name='HuggingFaceHub')['secret_token']
huggingface_hub.login(token=hf_api_key)

In [4]:
import boto3
import os
from pathlib import Path
import time

if False:
    s3_client = boto3.client('s3')
    bucket_name = 'sagemaker-research-methodology-extraction'
    timelogger = utils.TimeLogger()
    destination_dataset = 'SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'
    for subset in ['train', 'test', 'validation']:
        timelogger.log(f'START subset: {subset}')
        s3_folder_prefix = f'01_data/03_core/unified_works_{subset}'
        local_download_path = 'temp_files'
        utils.ensure_path(local_download_path)
    
        paginator = s3_client.get_paginator('list_objects_v2')
        file_entry_pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder_prefix)
    
        for file_entry_page in file_entry_pages:
            if 'Contents' in file_entry_page:
                for file_entry in file_entry_page['Contents']:
                    s3_key = file_entry['Key']
                    relative_path = Path(s3_key).relative_to(s3_folder_prefix)
                    path_in_repo = f'data/{subset}/{relative_path}.parquet'
                    print('path_in_repo', path_in_repo)
                    local_file_path = Path(local_download_path) / subset / relative_path
    
                    local_file_path.parent.mkdir(parents=True, exist_ok=True)
                    # print('relative_path.resolve()', str(local_file_path.resolve()), type(local_file_path.resolve()))
    
                    s3_client.download_file(bucket_name, s3_key, str(local_file_path))
                    # print(f"{subset}: Downloaded {s3_key} to {local_file_path}")
                    upload_result = huggingface_hub.upload_file(
                        path_or_fileobj=local_file_path,
                        path_in_repo=path_in_repo,
                        repo_id=destination_dataset,
                        repo_type="dataset",
                    )
                    # print(f"{subset}: Uploaded {s3_key} to {path_in_repo}")
                    if '/home/sagemaker-user/sagemaker_research_classification/src/04_transformation/temp_files/' in str(local_file_path.resolve()):
                        os.remove(local_file_path)
                        # print(f"{subset}: Deleted {s3_key} at {local_file_path}")
                    time.sleep(10)
        timelogger.log(f'END subset: {subset}')

In [10]:
from datasets import load_dataset, DatasetDict, Dataset
dataset_name = 'SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'
dataset = load_dataset(dataset_name, 'default')
dataset

README.md: 0.00B [00:00, ?B/s]

Using the latest cached version of the dataset since SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/sagemaker-user/.cache/huggingface/datasets/SteveAKopias___semantic_scholar_cs_full_text_with_open_alex_topics/default/0.0.0/4277fb6c72ba727c72d7504d852b958b6a97436c (last modified on Tue Sep 16 18:46:48 2025).


Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 310751
    })
    validation: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 38967
    })
    test: Dataset({
        features: ['id', 'topic_index', 'topic_display_name', 'subfield_index', 'subfield_display_name', 'title', 'abstract', 'fulltext', 'subset'],
        num_rows: 39017
    })
})

In [12]:
def filter_not_null(example, column_name):
    return example[column_name] is not None

In [18]:
dataset_title_subfieldindex = dataset.select_columns(['title', 'subfield_index'])
dataset_title_subfieldindex = dataset_title_subfieldindex.filter(lambda x: filter_not_null(x, 'title'))
dataset_title_subfieldindex = dataset_title_subfieldindex.filter(lambda x: filter_not_null(x, 'subfield_index'))
dataset_title_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'title_subfield', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics/commit/aa047a70d9ae6b554398ea952595097237c5ab96', commit_message='Upload dataset', commit_description='', oid='aa047a70d9ae6b554398ea952595097237c5ab96', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'), pr_revision=None, pr_num=None)

In [19]:
dataset_title_topicindex = dataset.select_columns(['title', 'topic_index'])
dataset_title_topicindex = dataset_title_topicindex.filter(lambda x: filter_not_null(x, 'title'))
dataset_title_topicindex = dataset_title_topicindex.filter(lambda x: filter_not_null(x, 'topic_index'))
# dataset_title_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_title_topic', num_proc=8)
dataset_title_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'title_topic', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics/commit/d587f52ec5833bbd6eeb11d65dd13a6f72d65f27', commit_message='Upload dataset', commit_description='', oid='d587f52ec5833bbd6eeb11d65dd13a6f72d65f27', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'), pr_revision=None, pr_num=None)

In [20]:
dataset_abstract_subfieldindex = dataset.select_columns(['abstract', 'subfield_index'])
dataset_abstract_subfieldindex = dataset_abstract_subfieldindex.filter(lambda x: filter_not_null(x, 'abstract'))
dataset_abstract_subfieldindex = dataset_abstract_subfieldindex.filter(lambda x: filter_not_null(x, 'subfield_index'))
# dataset_abstract_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_abstract_subfield', num_proc=8)
dataset_abstract_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'abstract_subfield', num_proc=8)

dataset_abstract_topicindex = dataset.select_columns(['abstract', 'topic_index'])
dataset_abstract_topicindex = dataset_abstract_topicindex.filter(lambda x: filter_not_null(x, 'abstract'))
dataset_abstract_topicindex = dataset_abstract_topicindex.filter(lambda x: filter_not_null(x, 'topic_index'))
# dataset_abstract_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_abstract_topic', num_proc=8)
dataset_abstract_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'abstract_topic', num_proc=8)

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

README.md: 0.00B [00:00, ?B/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

  2025-09-16T23:17:46.925451Z  WARN  Status Code: 504. Retrying..., request_id: ""
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:220



Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

Uploading the dataset shards (num_proc=8):   0%|          | 0/8 [00:00<?, ? shards/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics/commit/6e4e6688498883c6f0b8674a6d481bb9fb74a4b4', commit_message='Upload dataset', commit_description='', oid='6e4e6688498883c6f0b8674a6d481bb9fb74a4b4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'), pr_revision=None, pr_num=None)

In [23]:
dataset_fulltext_subfieldindex = dataset.select_columns(['fulltext', 'subfield_index'])
dataset_fulltext_subfieldindex = dataset_fulltext_subfieldindex.filter(lambda x: filter_not_null(x, 'fulltext'))
dataset_fulltext_subfieldindex = dataset_fulltext_subfieldindex.filter(lambda x: filter_not_null(x, 'subfield_index'))
# dataset_fulltext_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_fulltext_subfield')
dataset_fulltext_subfieldindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'fulltext_subfield', num_proc=8)

Uploading the dataset shards:   0%|          | 0/21 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   0%|          |  524kB /  235MB            

...

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :   2%|1         | 3.67MB /  209MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics/commit/8ae1ccebe3b0c5b79be723e3fdc84d9ce341147e', commit_message='Upload dataset', commit_description='', oid='8ae1ccebe3b0c5b79be723e3fdc84d9ce341147e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'), pr_revision=None, pr_num=None)

In [13]:
dataset_fulltext_topicindex = dataset.select_columns(['fulltext', 'topic_index'])
dataset_fulltext_topicindex = dataset_fulltext_topicindex.filter(lambda x: filter_not_null(x, 'fulltext'))
dataset_fulltext_topicindex = dataset_fulltext_topicindex.filter(lambda x: filter_not_null(x, 'topic_index'))
# dataset_fulltext_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'+'_fulltext_topic')
dataset_fulltext_topicindex.push_to_hub('SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', 'fulltext_topic', num_proc=1)

Uploading the dataset shards:   0%|          | 0/21 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  11%|#         | 24.8MB /  235MB            

...

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  16%|#5        | 33.3MB /  209MB            

CommitInfo(commit_url='https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics/commit/3e48a18e34575e20f2e58249ebeb169c985276de', commit_message='Upload dataset', commit_description='', oid='3e48a18e34575e20f2e58249ebeb169c985276de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics', endpoint='https://huggingface.co', repo_type='dataset', repo_id='SteveAKopias/SemanticScholarCSFullTextWithOpenAlexTopics'), pr_revision=None, pr_num=None)